In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
country_names = pd.read_csv('data/dl1_countrycodeorg_country_name.csv')
GBD_countries = sorted(country_names[country_names["country"].notna()]['Country Code'])
WB_countries = sorted(country_names[country_names["WBCountry"].notna()]['Country Code'])
print(len(GBD_countries), len(WB_countries))

204 216


In [3]:
sets_touse = []
# filelist = os.listdir('./')
# filelist = [file  for file in filelist if file.endswith('.csv')]
# filelist.remove('alpha.csv')
# filelist.remove('gd.csv')
# # filelist.remove('country_touse.csv')
# filelist = sorted(filelist)
# print(filelist)
PPP = True

filelist = ['GDP_ppp.csv', 'TC_ppp.csv', 'education_filled.csv', 'hepc_ppp.csv', 'laborparticipation_final.csv', 'physical_ppp.csv', 'population_un.csv', 'savings.csv']

savefile = 'country_touse_ppp.csv'
savefile_identify = 'country_indentify_ppp.csv'

df_identify = pd.DataFrame()
df_identify['Country Code'] = GBD_countries
df_identify = df_identify.merge(country_names[['Country Code', 'country', 'WBCountry', 'Region']], on='Country Code')
sets_touse = []
for file in filelist:
    df = pd.read_csv(os.path.join('data',file))
    countries = df['Country Code'].unique()
    df_identify[file] = df_identify['Country Code'].isin(countries)
    sets_touse.append(set(countries))
lens = [len(countries) for countries in sets_touse]
print(filelist, lens)
country_to_use = set.intersection(*sets_touse)
df = pd.DataFrame()
df['Country Code'] = sorted(list(country_to_use))
# df.to_csv(savefile,index=False)
# df_identify.to_csv(savefile_identify, index=False)
print(len(df))

['GDP_ppp.csv', 'TC_ppp.csv', 'education_filled.csv', 'hepc_ppp.csv', 'laborparticipation_final.csv', 'physical_ppp.csv', 'population_un.csv', 'savings.csv'] [187, 190, 216, 182, 283, 167, 201, 215]
143


In [4]:

df_missing = df_identify.copy()
df_missing = df_missing.replace(True, np.nan).set_index(['Region',  'country', 'WBCountry', 'Country Code'])
df_missing = df_missing.sort_index()
df_missing.dropna(inplace=True, how='all')
df_missing = df_missing.replace(False, 'X')

In [5]:
df_missing.to_csv('Missingdata.csv')

In [9]:
def get_table_by_region(df):
    projectStartYear, endyear = 2020, 2051
    discount, informal = 0.02, 0.11
    country_file = 'data/dl1_countrycodeorg_country_name.csv'
    countries = pd.read_csv(country_file)[['Country Code', 'Region', 'Income group']]
    countryinfofile = 'tmpresults/countryinfo.csv'
    countryinfo = pd.read_csv(countryinfofile)
    countryinfo['totalGDP'] = countryinfo['totalGDPD0'] * (discount == 0) \
                             + countryinfo['totalGDPD2'] * (discount == 0.02) \
                             + countryinfo['totalGDPD3'] * (discount == 0.03)
    
    df_table = df[(df['discount'] == discount) & (df['informal'] == informal)]
    df_table = df_table.merge(countries, on='Country Code').merge(countryinfo, on='Country Code')
    print ('confirms the lens of table ', len(df_table))  
    group1 = df_table.groupby(['Region', 'scenario']).sum().reset_index()
    group1['By regions'] = group1['Region'] 
    group2 = df_table.groupby(['Income group', 'scenario']).sum().reset_index()
    group2['By regions'] = group2['Income group'] 
    group3 = df_table.groupby('scenario').sum().reset_index()
    group3['By regions'] = 'Total'

    df1 = pd.concat([group1, group2, group3])
    df1['tax']  = df1['GDPloss'] / df1['totalGDP'] * 1000000000
    df1['pc_loss'] = df1['GDPloss'] / df1['totalPOP'] * (endyear - projectStartYear) * 1000000000

    df1['GDPloss'] = df1.apply(lambda x: format(round(x['GDPloss']), ','), axis=1)
    df1['tax'] = df1.apply(lambda x: format(x['tax'], '.3%'), axis=1)
    df1['pc_loss'] = df1.apply(lambda x: format(round(x['pc_loss']), ','), axis=1)
    df1 = df1[['By regions', 'scenario', 'GDPloss', 'tax', 'pc_loss']]

    val = df1[df1['scenario'] == 'val']
    lower = df1[df1['scenario'] == 'lower']
    upper = df1[df1['scenario'] == 'upper']
    df = val.merge(upper,on='By regions',suffixes=('', '_upper'))
    df= df.merge(lower,on='By regions',suffixes=('', '_lower'))

    data = df.copy()
    data['Economic cost in billions of 2017 INT$'] = data.apply(lambda x: '{}({}-{})'.format(x['GDPloss'], x['GDPloss_lower'], x['GDPloss_upper']), axis=1)
    data['Percentage of total GDP in 2020-2050'] = data.apply(lambda x: '{}({}-{})'.format(x['tax'], x['tax_lower'], x['tax_upper']), axis=1)
    data['Per capita loss in 2017 INT $'] = data.apply(lambda x: '{}({}-{})'.format(x['pc_loss'], x['pc_loss_lower'], x['pc_loss_upper']), axis=1)
    data = data[['By regions', 'Economic cost in billions of 2017 INT$', 'Percentage of total GDP in 2020-2050','Per capita loss in 2017 INT $']]
    data.to_csv('tables/Table2_i11d2.csv',index=False)
    return 0

In [12]:
import pandas as pd
df_input = pd.read_csv('results/aggregate_results_imputed.csv')
ConsiderTC,ConsiderMB,informal,discount = 1,1,0.11, 0.02
df = df_input[(df_input['discount']==discount)&
                (df_input['ConsiderTC']==ConsiderTC)&
                (df_input['ConsiderMB']==ConsiderMB)&
                (df_input['informal']==informal)]
get_table_by_region(df)

confirms the lens of table  612


0

GBD_countries